# Solution for SudoCV

_This challenge was based on a puzzlehunt-style puzzle I wrote with the title "Takeshi's Castle" and answer "The Cake is a Lie". I realised the sudoku had all the letters to make ETHICAL HACKS, so somehow or other this puzzle was born._

In [1]:
from z3 import *
from pwn import *

This is a thinly veiled sudoku puzzle that also doubles as a word search. For each phrase in the checklist, you get one point if the grid contains every word in that phrase (in any direction). The source code can be found in [sudocv.c](sudocv.c), but might be slightly obfuscated because it is compiled with `-O3`.

Anyway, let's start off by getting the set of 9-letters that are needed for the sudoku from the list of phrases.

In [2]:
phrases = ['attacks lattices', 'has stealth tactics', 'has the latest tech', 'hits the call stack', 'is leet', 'leaks assets', 'likes haskell', 'likes kali', 'sets a cheat sheet', 'takes shell access']
letterbank = ''.join({c for phrase in phrases for word in phrase.split() for c in word})
letterbank

'icseltkah'

Next, we create a dictionary of all tuples of positions in the grid that correspond to a valid wordsearch-style word.

In [3]:
dic = {i:set() for i in range(10)}
for start in range(81):
    for dx in [-1,0,1]:
        for dy in [-1,0,1]:
            if dx or dy:
                arr = []
                for n in range(9):
                    if 0 <= start%9 + dx*n < 9 and 0 <= start//9 + dy * n < 9:
                        arr.append(start + (dy*9+dx)*n)
                        dic[len(arr)].add(tuple(arr))

Now we just have to solve the sudoku. We will use z3 to solve this, denoting each element as an integer between 0 and 8 inclusive (for easier indexing).

In [4]:
s = Solver()
iv = IntVector('i', 81)
s.add([And(i >= 0, i < 9) for i in iv])

for i in range(9):
    s.add(Distinct([iv[n] for n in range(81) if n//9 == i])) # distinct elements in rows
    s.add(Distinct([iv[n] for n in range(81) if n%9 == i])) # distinct elements in columns
    s.add(Distinct([iv[n] for n in range(81) if n//27*3+n%9//3 == i])) # distinct elements in group
    
def has_word(w):
    ns = [letterbank.index(c) for c in w]
    return Or([And([iv[p]==n for (p,n) in zip(ps, ns)]) for ps in dic[len(w)]])

def has_phrase(p):
    return And([has_word(w) for w in p.split()])

score = sum([If(has_phrase(p), 1, 0) for p in phrases])
s.add(score >= 5)

while s.check() == sat:
    sudocv = ''.join(letterbank[s.model()[i].as_long()] for i in iv)
    print(sudocv)
    s.add(Or([i != s.model()[i] for i in iv]))

ekslichtatclhkaesiiahtseklcctiklhaeshseiatcklalkcestihleashkictshtecilakkicatlshe
athcilskeiseakhlctclkesthaiseahlkitclkctaieshhitsecklatcikhsaelkalicethsehsltacik
etichalskkcatslehislhiekatclhtkicseaikslaehctcaehtskilhekactilstslekicahaicslhtke
ehsltacikkalicethstcikhsaelhitsecklalkctaieshseahlkitcclkesthaiiseakhlctathcilske
kicatlsheshtecilakleashkictalkcestihhseiatcklctiklhaesiahtseklctclhkaesiekslichta
ekthlsciahacikelstslitcakehlikstheactchealskiaescikthlctakeihlsihelstackkslahcite
aicslhtketslekicahhekactilscaehtskilikslaehctlhtkicseaslhiekatckcatslehietichalsk
kslahciteihelstackctakeihlsaescikthltchealskilikstheacslitcakehhacikelstekthlscia


Great success! It turns out that there are exactly possible 8 solutions. We really only need (any) one of them, but it probably doesn't hurt to be exhaustive. In this case, they are all actually just rotations and reflections of a single unique solution. We draw one such grid here, with word search matches.

![grid](sudocvgrid.png)

And that's all there is to it. We can pick any of the eight solutions and copy and paste it into the executable (they all give the same flag), but for demonstrative purposes we use pwntools here.

In [5]:
with process('./sudocv') as sh:
    sh.sendline(sudocv.encode())
    print(sh.recvall().decode())

[x] Starting local process './sudocv'
[+] Starting local process './sudocv': pid 2935
[x] Receiving all data
[x] Receiving all data: 0B
[*] Process './sudocv' stopped with exit code 0 (pid 2935)
[x] Receiving all data: 371B
[+] Receiving all data: Done (371B)
Please submit your (sudo)cv:
Scanning against the ethical hacks checklist:
[ ] attacks lattices
[ ] has stealth tactics
[X] has the latest tech
[X] hits the call stack
[X] is leet
[ ] leaks assets
[X] likes haskell
[X] likes kali
[ ] sets a cheat sheet
[ ] takes shell access

Candidate score: 5/10.
You PASS! Here is your flag:
SEE{the_cake_is_a_lie_at_takeshis_castle}

